# Laboratorio 4
- Juan Pablo Solis
- Brandon Reyes
- Carlos Valladares

# Task 1 - Preguntas Teoricas
1. Explique con sus propias palabras, qué son los grafos computaciones y cuál es su importancia para el
cálculo de gradientes en aplicaciones como backpropagation.
    - Un grafo computacional es un tipo de grafico el cual su nodo raiz representa la expresión matemática final y cada nodo representa subexpresiones intermedias
2. Detalle cuales son los componentes y pasos que conforman una red neuronal. Con esto en mente, ¿cómo
mejoraría el perceptrón que hizo en el laboratorio pasado?
    - Neuronas: Unidades básicas que procesan información a través de funciones de activación.
    - Capas: Incluyen capa de entrada, capas ocultas y capa de salida.
    - Pesos y sesgos: Factores que determinan la influencia de una neurona en otra.
    - Función de activación: Introduce no linealidad en la red (ReLU, Sigmoide, etc.).
    - Función de costo: Mide la diferencia entre la salida esperada y la obtenida.
    - Algoritmo de optimización: Como el descenso de gradiente para actualizar los pesos.
    - Para mejorar el perceptron añadiendo más capas ocultas y utilizando funciones de activación no lineales
3. Investigue y explique cómo se selecciona el valor K usando el método de la silueta para el algoritmo de K-
Means
    - El método del coeficiente de silueta mide la calidad de los clusters evaluando la similitud de cada punto con su propio cluster y comparándola con otros clusters
    - La ecuacion es S(i)= (b(i)−a(i))/(max(a(i),b(i)))
    - Donde a(i) es la distancia promedio del punto i a los demás puntos de su mismo cluster y b(i) es la menor distancia promedio del punto i a los puntos de otro cluster.
4. ¿Cómo podría ayudarle el PCA a mejorar la calidad de sus clusters cuando se usa K-Means?
    - El PCA ayuda a reducir la dimensionalidad de los datos preservando la varianza más significativa, por lo que puede mejorar significativamente los resultados de K-Means al facilitar la identificación de patrones relevantes en los datos
​


# Task 2.1 K-Means y Segmentación de Clientes de Banco

## 2.1.0 Exploracion e Importacion de datos

In [13]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample


In [ ]:
filepath = "bank_transactions.csv"
df = pd.read_csv(filepath)
#Exploracion de datos - Inciso 2.0
print("Distribución inicial de la columna 'Genero':")
print(df['CustGender'].value_counts())
print("\nVerificacion de datos nulos")
print(df.isnull().sum())
print("\nResumen estadistico de los datos")
print(df.describe())




Distribución inicial de la columna 'Genero':
CustGender
M    765530
F    281936
T         1
Name: count, dtype: int64

Verificacion de datos nulos
TransactionID                 0
CustomerID                    0
CustomerDOB                3397
CustGender                 1100
CustLocation                151
CustAccountBalance         2369
TransactionDate               0
TransactionTime               0
TransactionAmount (INR)       0
dtype: int64

Resumen estadistico de los datos
       CustAccountBalance  TransactionTime  TransactionAmount (INR)
count        1.046198e+06     1.048567e+06             1.048567e+06
mean         1.154035e+05     1.570875e+05             1.574335e+03
std          8.464854e+05     5.126185e+04             6.574743e+03
min          0.000000e+00     0.000000e+00             0.000000e+00
25%          4.721760e+03     1.240300e+05             1.610000e+02
50%          1.679218e+04     1.642260e+05             4.590300e+02
75%          5.765736e+04     2.000100e+05

In [15]:
# Eliminar filas con valores nulos en las columnas especificadas
df = df.dropna(subset=["CustomerDOB", "CustGender", "CustLocation", "CustAccountBalance"])

print("\nVerificacion de datos nulos")
print(df.isnull().sum())


Verificacion de datos nulos
TransactionID              0
CustomerID                 0
CustomerDOB                0
CustGender                 0
CustLocation               0
CustAccountBalance         0
TransactionDate            0
TransactionTime            0
TransactionAmount (INR)    0
dtype: int64


In [16]:
# Eliminar filas donde el género sea 'T'
df = df[df["CustGender"] != "T"]

label_encoder = LabelEncoder()

# Aplicar Label Encoding a la columna 'CustGender' (M -> 1, F -> 0)
df["CustGender"] = label_encoder.fit_transform(df["CustGender"])

# Aplicar Label Encoding a la columna 'CustLocation'
df["CustLocation"] = label_encoder.fit_transform(df["CustLocation"])



In [17]:

# Seleccionar solo las primeras columnas para mostrar
df_head = df.iloc[:, :5]  # Tomamos las primeras 5 columnas

# Mostrar los datos
df_head


,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation
0,T1,C5841053,10/1/94,0,3557
1,T2,C2142763,4/4/57,1,3618
2,T3,C4417068,26/11/96,0,5219
3,T4,C5342380,14/9/73,0,5219
4,T5,C9031234,24/3/88,0,5606
...,...,...,...,...,...
1048562,T1048563,C8020229,8/4/90,1,5738
1048563,T1048564,C6459278,20/2/92,1,5580
1048564,T1048565,C6412354,18/5/89,1,3367
1048565,T1048566,C6420483,30/8/78,1,9059


In [19]:
# Contar la cantidad de registros por género (ahora en formato numérico)
gender_counts = df["CustGender"].value_counts()
min_class = gender_counts.min()  # Número de registros en la clase minoritaria

# Separar por género usando los valores numéricos
df_male = df[df["CustGender"] == 1]  # M
df_female = df[df["CustGender"] == 0]  # F

# Balancear mediante undersampling (reducimos la clase mayoritaria al tamaño de la minoritaria)
df_male_balanced = resample(df_male, replace=False, n_samples=min_class, random_state=42)
df_female_balanced = resample(df_female, replace=False, n_samples=min_class, random_state=42)

# Combinar ambos para formar el dataset balanceado
df_balanced = pd.concat([df_male_balanced, df_female_balanced])

# Mezclar las filas aleatoriamente para evitar sesgos
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

print(df_balanced['CustGender'].value_counts())


CustGender
0    280635
1    280635
Name: count, dtype: int64


2.1.1 Entrenamiento de Modelo K-Means

In [ ]:
import random
import matplotlib.pyplot as plt

# Función para calcular la distancia euclidiana
def euclidean_distance(point1, point2):
    return sum((p1 - p2) ** 2 for p1, p2 in zip(point1, point2)) ** 0.5

# Función de K-Means desde cero
def k_means_custom(data, k, max_iters=100):
    # Inicializar centroides de manera aleatoria
    centroids = random.sample(data, k)

    for _ in range(max_iters):
        clusters = {i: [] for i in range(k)}  # Diccionario de clusters

        # Asignar cada punto al cluster más cercano
        for point in data:
            distances = [euclidean_distance(point, centroid) for centroid in centroids]
            cluster_idx = distances.index(min(distances))
            clusters[cluster_idx].append(point)

        # Calcular nuevos centroides
        new_centroids = []
        for i in range(k):
            cluster_points = clusters[i]
            if cluster_points:  # Evitar centroides vacíos
                new_centroid = [sum(dim) / len(dim) for dim in zip(*cluster_points)]
            else:
                new_centroid = random.choice(data)  # Reasignar aleatoriamente si el cluster está vacío
            new_centroids.append(new_centroid)

        # Si los centroides no cambian, detener iteraciones
        if new_centroids == centroids:
            break
        centroids = new_centroids

    return clusters, centroids

# Selección de características para clustering
feature_1 = "CustAccountBalance"
feature_2 = "TransactionAmount (INR)"
data_points = df_balanced[[feature_1, feature_2]].values.tolist()

# Determinar el número óptimo de clusters usando el método del codo
inertia_values = []
k_values = range(1, 10)  # Probamos con k de 1 a 10

for k in k_values:
    clusters, centroids = k_means_custom(data_points, k)
    inertia = sum(sum(euclidean_distance(point, centroids[idx])**2 for point in clusters[idx]) for idx in clusters)
    inertia_values.append(inertia)

# Graficar el método del codo
plt.figure(figsize=(8, 5))
plt.plot(k_values, inertia_values, marker="o", linestyle="-")
plt.xlabel("Número de Clusters (k)")
plt.ylabel("Inercia (Suma de Distancias al Cuadrado)")
plt.title("Método del Codo para Selección de k")
plt.show()

# Seleccionamos el número óptimo de clusters (usaremos k=3 basado en el gráfico)
optimal_k = 3
clusters, centroids = k_means_custom(data_points, optimal_k)

# Convertir clusters a formato visualizable
cluster_points = []
for i in range(optimal_k):
    cluster_points.append([list(point) for point in clusters[i]])

# Graficar clusters
colors = ["red", "blue", "green", "orange", "purple", "brown"]
plt.figure(figsize=(8, 6))

for i in range(optimal_k):
    cluster_x = [point[0] for point in cluster_points[i]]
    cluster_y = [point[1] for point in cluster_points[i]]
    plt.scatter(cluster_x, cluster_y, color=colors[i], label=f"Cluster {i+1}")

# Graficar centroides
centroid_x = [centroid[0] for centroid in centroids]
centroid_y = [centroid[1] for centroid in centroids]
plt.scatter(centroid_x, centroid_y, color="black", marker="X", s=200, label="Centroides")

plt.xlabel(feature_1)
plt.ylabel(feature_2)
plt.title("Segmentación de Clientes con K-Means (k=3)")
plt.legend()
plt.show()

# Justificación de las características seleccionadas
justificacion = """
Las características seleccionadas son:
1. **CustAccountBalance (Saldo en cuenta)**: Representa la capacidad financiera del cliente y puede indicar patrones de comportamiento.
2. **TransactionAmount (INR) (Monto de transacción)**: Refleja el nivel de actividad del cliente en términos de compras o movimientos financieros.

Estas características son clave para segmentar clientes porque pueden ayudar a identificar:
- Clientes de alto valor con grandes saldos y transacciones elevadas.
- Clientes regulares con balances y transacciones moderadas.
- Clientes con actividad baja o esporádica.

El uso de estas dos variables nos permite visualizar los grupos de clientes de manera efectiva en un gráfico 2D.
"""

justificacion
